# Figure 9

## Imports

In [ ]:
import dolfin
import matplotlib.pyplot as plt
import meshio
import numpy

import dolfin_mech as dmech
import get_invariants

## Parameters

### Mesh

In [ ]:
cube_params = {"path_and_mesh_name":"Meshes/generic_lung.xdmf"}

### Material

In [ ]:
params = {
    "alpha": 0.16, # MG: Units!
    "gamma":0.5, # MG: Units!
    "c1":0.6, # MG: Units!
    "c2":0., # MG: Units!
    "kappa":1e2, # MG: Units!
    "eta":1e-5, # MG: Units!
    "rho_solid":1.06e-6} # MG: Units!

mat_params = {"scaling":"linear", "parameters":params}

## Loading

In [ ]:
pe, pi = -0.5, -2. # MG: Units!

g = 9.81e3 # MG: Units!

## Computing strain fields in prone and supine positions

In [ ]:
invariants = {}

gravity_lst = [-1.,+1.]
for gravity_cste in gravity_lst:

    load_params_inverse = {
        "type":"p_boundary_condition0", "f": gravity_cste*9.81e3, "P0":float(pe)}
    load_params_direct_exhalation = {
        "type":"p_boundary_condition", "f": gravity_cste*9.81e3, "P0":float(pe)}
    load_params_direct_inhalation = {
        "type":"p_boundary_condition", "f": gravity_cste*9.81e3, "P0":float(pi)}

    ### computing the unloaded configuration
    Uref, phisref_computation, V_expi_ini, V_unloaded = dmech.run_RivlinCube_PoroHyperelasticity(
        inverse=1,
        cube_params=cube_params,
        mat_params=mat_params,
        load_params=load_params_inverse,
        inertia=1,
        step_params={"dt_ini":1., "dt_min":1e-4},
        res_basename="Fig9-unloaded",
        get_results=1,
        verbose=1)

    ### computing the end-exhalation configuration
    phisref_imposed = [numpy.random.uniform(low=0.4, high=0.6) for i in range(len(phisref_computation))]
    U_exhal, phisexhal_g, V_unloaded, V_exhal = dmech.run_RivlinCube_PoroHyperelasticity(
        inverse=0,
        cube_params=cube_params,
        move_params={"move":True, "U":Uref},
        porosity_params={"type": "function_xml_from_array", "val":phisref_imposed},
        mat_params=mat_params,
        load_params=load_params_direct_exhalation,
        inertia=1,
        step_params={"dt_ini":0.125, "dt_min":1e-4},
        res_basename="Fig9-exhalation",
        get_results=1,
        verbose=1)

    U_inhal, phisinhal, V_unloaded, Vinhal = dmech.run_RivlinCube_PoroHyperelasticity(
        inverse=0,
        cube_params=cube_params,
        move_params={"move":True, "U":Uref},
        porosity_params={"type": "function_xml_from_array", "val":phisref_imposed},
        mat_params=mat_params,
        load_params=load_params_direct_inhalation,
        inertia=1,
        step_params={"dt_ini":0.125, "dt_min":1e-4},
        res_basename="Fig9-inhalation",
        get_results=1,
        verbose=1)

    mesh = dolfin.Mesh()
    mesh_name = str(cube_params["path_and_mesh_name"])
    dolfin.XDMFFile(mesh_name).read(mesh)
    dolfin.ALE.move(mesh, Uref)

    invariants_mesh = get_invariants.get_invariants(U_exhal=U_exhal, U_inhal=U_inhal, mesh=mesh, lognorm=False)
    invariants["g="+str(gravity_cste)] = invariants_mesh

## Post-processing

### Reading displacement field from Patte et al. and computing associated invariants

In [ ]:
mesh = dolfin.Mesh()
dolfin.XDMFFile("Meshes/mesh_patte.xdmf").read(mesh)
fe = dolfin.VectorElement(
    family="CG",
    cell=mesh.ufl_cell(),
    degree=1)
U_fs= dolfin.FunctionSpace(
    mesh,
    fe)

U_inhal_patte = dolfin.Function(U_fs, name="U")
U_exhal_patte = dolfin.Function(U_fs, name="U")

mesh_meshio = meshio.read("Meshes/mesh_patte.xdmf")
u_meshio = mesh_meshio.point_data["U"]
u_meshio = u_meshio.tolist()
u_meshio = [item for sublist in u_meshio for item in sublist] 

c = 0
for dof in dolfin.vertex_to_dof_map(U_fs):
    U_inhal_patte.vector()[dof] = u_meshio[c]
    U_exhal_patte.vector()[dof] = 0.
    c += 1

In [ ]:
invariants_mesh_patte = get_invariants.get_invariants(U_exhal=U_exhal_patte, U_inhal=U_inhal_patte, mesh=mesh, lognorm=False)

### Generating plots

#### I1

In [ ]:
fig, ax = plt.subplots()
plt.plot(invariants["g=-1"]["zone"]   , invariants["g=-1"]["I1"]   , color="red"  , linestyle="-", marker="s", label="Prone position" )
plt.plot(invariants["g=1"]["zone"]    , invariants["g=1"]["I1"]    , color="blue" , linestyle="-", marker="s", label="Supine position")
plt.plot(invariants_mesh_patte["zone"], invariants_mesh_patte["I1"], color="black", linestyle="-", marker="s", label="[Patte et al.]" )
ax.set_ylabel("I1")
ax.set_title("zone")
plt.legend(loc="upper right")
plt.grid()
plt.show()

#### I2

In [ ]:
fig, ax = plt.subplots()
plt.plot(invariants["g=-1"]["zone"]   , invariants["g=-1"]["I2"]   , color="red"  , linestyle="-", marker="s", label="Prone position" )
plt.plot(invariants["g=1"]["zone"]    , invariants["g=1"]["I2"]    , color="blue" , linestyle="-", marker="s", label="Supine position")
plt.plot(invariants_mesh_patte["zone"], invariants_mesh_patte["I2"], color="black", linestyle="-", marker="s", label="[Patte et al.]" )
ax.set_ylabel("I2")
ax.set_title("zone")
plt.legend(loc="upper right")
plt.grid()
plt.show()

#### J

In [ ]:
fig, ax = plt.subplots()
plt.plot(invariants["g=-1"]["zone"]   , invariants["g=-1"]["J"]   , color="red"  , linestyle="-", marker="s", label="Prone position" )
plt.plot(invariants["g=1"]["zone"]    , invariants["g=1"]["J"]    , color="blue" , linestyle="-", marker="s", label="Supine position")
plt.plot(invariants_mesh_patte["zone"], invariants_mesh_patte["J"], color="black", linestyle="-", marker="s", label="[Patte et al.]" )
ax.set_ylabel("J")
ax.set_title("zone")
plt.legend(loc="upper right")
plt.grid()
plt.show()